In [ ]:
# https://www.bp.com/en/global/corporate/energy-economics/statistical-review-of-world-energy.html

import openpyxl

xlsx = openpyxl.load_workbook("bp-stats-review-2021-all-data.xlsx")

In [ ]:
for index, value in enumerate(xlsx.sheetnames):
    if (
        value != "Contents"
        and type(xlsx.worksheets[type_index].cell(3, 2).value) == int
    ):
        print(f"{value} [{xlsx.worksheets[index].cell(3, 1).value}]")
        # print(f'{value} [{xlsx.worksheets[index].cell(3, 2).value}]')

In [ ]:
# Type
types = []
for index, value in enumerate(xlsx.sheetnames):

    # if value not in ['Contents', 'Approximate conversion factors', 'Definitions', 'Methodology'] \
    #        and type(xlsx.worksheets[type_index].cell(3, 2).value) == int:

    if xlsx.worksheets[index].cell(3, 1).value in [
        "Exajoules",
        "Petajoules",
        "Exajoules (input-equivalent)",
        "Terawatt-hours",
    ]:
        values_split = value.split(" ")
        if "Generation" in values_split:
            types.append({"label": value, "index": index})
        if "Capacity" in values_split:
            types.append({"label": value, "index": index})
        if "Consumption" in values_split:
            types.append({"label": value, "index": index})
for t in types_:
    print(t["label"])

In [ ]:
import re

import pandas as pd

values = {
    "values": [],
    "type": [],
    "unit": [],
    "type unit": [],
    "year": [],
    "region": [],
}
for type_ in types:
    type_index = type_["index"]
    type_label = type_["label"]
    # Year
    years = []
    index = 2
    re_ = re.compile("^[0-9]+$")
    while True:
        value = xlsx.worksheets[type_index].cell(3, index).value
        if xlsx.worksheets[type_index].cell(2, index).value is not None:
            break
        years.append({"label": value, "index": index})
        index += 1

    # Country
    regions = []
    index = 5
    while True:
        value = xlsx.worksheets[type_index].cell(index, 1).value
        if value is not None and value == "Switzerland":
            regions.append({"label": value, "index": index})
        if value == "Total World":
            break
        index += 1

    for year in years:
        for region in regions:
            value = (
                xlsx.worksheets[type_index].cell(region["index"], year["index"]).value
            )
            if type_label.endswith(" - TWh"):
                type_label = type_label[:-6]
            if type_label.endswith(" - EJ"):
                type_label = type_label[:-5]
            if type_label.endswith(" - PJ"):
                type_label = type_label[:-5]

            unit = xlsx.worksheets[type_index].cell(3, 1).value
            if unit == "Terawatt-hours":
                if value is not None and type(value) != str:
                    value = value * 3.6
                unit = "Petajoules"
            if unit == "Exajoules":
                if value is not None and type(value) != str:
                    value = value * 1000
                unit = "Petajoules"
            if unit == "Exajoules (input-equivalent)":
                if value is not None and type(value) != str:
                    value = value * 1000
                unit = "Petajoules (input-equivalent)"

            values["values"].append(value)
            values["year"].append(year["label"])
            values["region"].append(region["label"])
            values["type"].append(type_label)
            values["unit"].append(unit)
            values["type unit"].append(f"{type_label} [{unit}]")

df = pd.DataFrame(values)
df

In [ ]:
import plotly.express as px

fig = px.line(df, x="year", y="values", color="type unit", width=800, height=800)
fig.update_layout(
    yaxis_type="log",
    xaxis={
        "rangeslider": {"visible": True},
    },
)
fig.show()